# PDF에서 문장이 이상하게 잘린 경우 문장 교종해주는 코드
190903
최종

# 잘린 첫글자가 숫자인 경우는 안됨....
1. 여러 문장이 한줄에 입력된 경우 잘라주기
2. 소문자, 대문자, 숫자로 시작하는 문장의 index
3. PDF로 문장이 짤린 경우 이어주기
4. txt로 저장('~_fixed.txt')

In [1]:
import os, glob, re
from nltk import tokenize

In [2]:
## function

def split_long_one_line(total_line):
    ## 1. 여러 문장이 한줄에 입력된 경우 잘라주기
    fixed_long_line = []
    for long_line in total_line:
        # spacy 문장 나누기로 쪼개서 문장별 correction 해서 저장
        text_sentences = tokenize.sent_tokenize(long_line)
        # 쪼개진 문장 갯수
        for idx, sentence in enumerate(text_sentences):
            len_sentences = idx + 1
        # 쪼개질게 없으면 바로 통과
        if len_sentences == 1:
            fixed_long_line.append(long_line)
        elif len_sentences > 1:
            for sentence in text_sentences:
                fixed_long_line.append(sentence) # 잘린건 뒤에 '\n가 없어서 붙여줘야한다'
    return fixed_long_line

def merge_sentences(fixed_long_line):
    ## 2. 소문자, 대문자, 숫자로 시작하는 문장의 index
    lower_idx, upper_idx, not_eng_idx = [], [], []
    for line_idx, line in enumerate(fixed_long_line):
        if line[0].islower():
            lower_idx.append(line_idx)
        elif line[0].isupper():
            upper_idx.append(line_idx)
        else:
            not_eng_idx.append(line_idx)

    ## 3. PDF로 문장이 짤린 경우 이어주기
    fixed_short_line = []
    count = 0
    while count < len(fixed_long_line)-1:
        # 시작이 대문자고 다음이 소문자면
        if fixed_long_line[count][0].isupper() and fixed_long_line[count+1][0].islower():
            # 소문자가 몇개 연달아 나오는지 센다
            count_lower = 0
            while 1:
                if count+count_lower+1 in lower_idx:
                    count_lower += 1
                else:
                    break
            # 대문자로 시작하는 문장
            plus_line = fixed_long_line[count].strip()+' '  # 좌우 자르고 이어주기
            # 대문자 뒤 소문자 연속하는 부분 idx for문
            for ii in range(count+1, count+ count_lower+1):
                plus_line += fixed_long_line[ii].strip()+' ' # 좌우 자르고 이어주기

            # spacy 문장 나누기로 쪼개서 문장별 correction 해서 저장
            text_sentences = tokenize.sent_tokenize(plus_line)
            for sentence in text_sentences:
                fixed_short_line.append(sentence.strip())
            count += count_lower+1
        else:
            # '\n' 후에 '\n' 나오면 다음줄 건너띄자
            if fixed_long_line[count] == '\n' and fixed_long_line[count+1] == '\n':
                fixed_short_line.append(fixed_long_line[count].strip())
                count += 2
            else:
                fixed_short_line.append(fixed_long_line[count].strip())
                count += 1
    return fixed_short_line

def save_fixed_as_txt(path, fixed_short_line):
    ## 4. txt로 저장
    f = open(path.split('.')[0]+'_fixed.txt', 'w', encoding='UTF-8')
    for fixed_line in fixed_short_line:
        # 4글자 미만은 버리자. 0은 '\n'을 살리기 위해
        if len(fixed_line) == 0 or len(fixed_line) > 3:
            f.write(fixed_line+'\n')  # 왼쪽만 공백 제거,  strip으로 하면 '\n'도 날아간다
    f.close()
    
def sentence_corrector(report, src_path, dst_path):  # path, dst_path):
    # 데이타 읽어오기
    path     = os.path.join(src_path, report)
    path_save= os.path.join(dst_path, report)
    test_data = open(path, 'r', encoding='UTF-8').readlines()
    total_line = []
    for line in test_data:
        ## 정규표현식으로 에러나는 부분 교체해주기
        # “”는 문장쪼개기에서 에러나므로 '"' 로 치환
        line = re.sub(r'[“”]', '"', line)
        line = re.sub(r'[\xa0]', '' , line)  # 이상한 빈칸 없애기 
        line = re.sub(r' . .'  , '' , line)  # . . . . .3 목차 지우기
        total_line.append(line)
    ## 1. 여러 문장이 한줄에 입력된 경우 잘라주기
    fixed_long_line = split_long_one_line(total_line)
    
    ## 2. 소문자, 대문자, 숫자로 시작하는 문장의 index
    ## 3. PDF로 문장이 짤린 경우 이어주기
    fixed_short_line = merge_sentences(fixed_long_line)
    ## 4. txt로 저장
    save_fixed_as_txt(path_save, fixed_short_line)
    
    
def make_dir(dirname):
    if not os.path.isdir(dirname):
        os.mkdir(dirname)


In [4]:
src_path = '4_KP_데이터 추출완료_정제전_394_개행처리/txt'
dst_path = src_path + '_sentence/'
target = '.txt'

make_dir(dst_path)
report_list = os.listdir(src_path)
report_list[:5]

['AdobeFlashPlayer0day-CVE-2016-1010.txt',
 'Ammyy.txt',
 'Analysis_and_detection_guidance_for_CVE-2019-0708.txt',
 'Analysis_and_evolution_of_Spring_Dragon_tools.txt',
 'Analysis_of_alleged_Supermicro_supply_chain_attack.txt',
 'Analysis_of_ANEL_shows_a_potential_connection_between_APT10_and_BlueTermite.txt',
 'Analysis_of_Redleaves_variants_used_by_APT10.txt',
 'Analysis_of_Shadowbrokers_leak_and_links_to_Equation_Group.txt',
 'Analysis_of_the_Oilrig_data_dump.txt',
 'Andariel_targets_weblogic_server,_deploys_new_signed_malware.txt']

In [5]:
for report_idx, report in enumerate(report_list):
    print('%d / %d'% (report_idx+1, len(report_list)))
    ## !! sentence corrector 실행!!
    sentence_corrector(report, src_path, dst_path)
    
print('Done')

1 / 393
2 / 393
3 / 393
4 / 393
5 / 393
6 / 393
7 / 393
8 / 393
9 / 393
10 / 393
11 / 393
12 / 393
13 / 393
14 / 393
15 / 393
16 / 393
17 / 393
18 / 393
19 / 393
20 / 393
21 / 393
22 / 393
23 / 393
24 / 393
25 / 393
26 / 393
27 / 393
28 / 393
29 / 393
30 / 393
31 / 393
32 / 393
33 / 393
34 / 393
35 / 393
36 / 393
37 / 393
38 / 393
39 / 393
40 / 393
41 / 393
42 / 393
43 / 393
44 / 393
45 / 393
46 / 393
47 / 393
48 / 393
49 / 393
50 / 393
51 / 393
52 / 393
53 / 393
54 / 393
55 / 393
56 / 393
57 / 393
58 / 393
59 / 393
60 / 393
61 / 393
62 / 393
63 / 393
64 / 393
65 / 393
66 / 393
67 / 393
68 / 393
69 / 393
70 / 393
71 / 393
72 / 393
73 / 393
74 / 393
75 / 393
76 / 393
77 / 393
78 / 393
79 / 393
80 / 393
81 / 393
82 / 393
83 / 393
84 / 393
85 / 393
86 / 393
87 / 393
88 / 393
89 / 393
90 / 393
91 / 393
92 / 393
93 / 393
94 / 393
95 / 393
96 / 393
97 / 393
98 / 393
99 / 393
100 / 393
101 / 393
102 / 393
103 / 393
104 / 393
105 / 393
106 / 393
107 / 393
108 / 393
109 / 393
110 / 393
111 / 39